In [1]:
import numpy as np
import os
import math
import random
import torch
from sklearn.model_selection import train_test_split
from RNN import RNN
from tqdm import tqdm_notebook

### Prepare the data

In [2]:
EVAL_DATA_DIR = os.path.abspath(os.path.abspath('') + '/Aniyama_groundtruth')

In [3]:
data_types = [p for p in os.listdir(EVAL_DATA_DIR) if not p.startswith('.')]
dataset = []
for t in data_types:
    type_dir = f'{EVAL_DATA_DIR}/{t}'
    for file in os.listdir(type_dir):
        filename = f'{type_dir}/{file}'
        examples = []
        with open(filename, 'r') as f:
            for i, line in enumerate(f.readlines()):
                if i == 0:
                    continue
                tokens = line.rstrip().split(',')
                power, anomaly = float(tokens[1]), int(tokens[2])
                example = [power, anomaly]
                examples.append(example)
    examples = np.array(examples)
    dataset.append(examples)

In [4]:
data = []
labels = []
for chunk in dataset:
    size = chunk.shape[0]
    new_size = (math.ceil(size / 10) * 10)
    pad_size = new_size - size
    padding = np.zeros((pad_size, 2))
    new_chunk = np.vstack((chunk, padding)).reshape(new_size // 10, 10, 2)
    for i in range(new_chunk.shape[0]):
        data.append(new_chunk[i,:,0])
        label = new_chunk[i,:,1].sum() > 0
        labels.append(float(label))
data = np.array(data)
labels = np.array(labels)

In [5]:
data.shape, labels.shape

((3429, 10), (3429,))

In [6]:
X_train, X_test, y_train, y_text = train_test_split(data, labels, test_size = 0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train, X_valid, X_test = torch.FloatTensor(X_train), torch.FloatTensor(X_train), torch.FloatTensor(X_train)
y_train, y_valid, y_test = torch.FloatTensor(y_train), torch.FloatTensor(y_train), torch.FloatTensor(y_train)

In [8]:
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, data, labels):
        'Initialization'
        self.labels = labels
        self.data = data

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        return self.data[index], self.labels[index]

In [28]:
config = {
    'batch_size': 32,
    'shuffle': True,
    'num_workers': 3,
    'drop_last': True,
    'num_epochs': 100,
    'encode_dim': 10,
    'hidden_dim': 128,
    'output_dim': 32,
    'num_layers': 2,
    'dropout': 0.3,
    'device': device
}

In [36]:
train_set = Dataset(X_train, y_train)
train_loader = data.DataLoader(
    train_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
    num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

validation_set = Dataset(X_valid, y_valid)
validation_loader = data.DataLoader(
    validation_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
    num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

test_set = Dataset(X_test, y_test)
test_loader = data.DataLoader(
    test_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
    num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

model = RNN(
    config['encode_dim'],
    config['hidden_dim'],
    config['output_dim'],
    config['num_layers'],
    config['dropout']
)

In [37]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 207,904 trainable parameters


In [38]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()

model = model.to(config['device'])
criterion = criterion.to(config['device'])

In [39]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [40]:
def train(model, loader, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_data, batch_labels in loader:
        
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
                
        optimizer.zero_grad()
        
        predictions = model(batch_data).squeeze(0)
        
        loss = criterion(predictions, batch_labels)
        
        acc = binary_accuracy(predictions, batch_labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [41]:
def evaluate(model, loader, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch_data, batch_labels in loader:
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            predictions = model(batch_data).squeeze(0)
            
            loss = criterion(predictions, batch_labels)
            
            acc = binary_accuracy(predictions, batch_labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [42]:
def test(model, loader, device):
    epoch_acc = 0
    with torch.no_grad():
    
        for batch_data, batch_labels in loader:
            batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
            
            predictions = model(batch_data).squeeze(0)
            acc = binary_accuracy(predictions, batch_labels)

            epoch_acc += acc.item()
        
    return epoch_acc / len(loader)

In [43]:
best_valid_loss = float('inf')

for epoch in tqdm_notebook(range(config['num_epochs'])):
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, config['device'])
    valid_loss, valid_acc = evaluate(model, validation_loader, criterion, config['device'])
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'rnn-base-model.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print('\t-------------------------------------')

	Train Loss: 0.698 | Train Acc: 47.56%
	 Val. Loss: 0.696 |  Val. Acc: 50.34%
	-------------------------------------------
	Train Loss: 0.695 | Train Acc: 49.71%
	 Val. Loss: 0.699 |  Val. Acc: 48.78%
	-------------------------------------------
	Train Loss: 0.697 | Train Acc: 49.07%
	 Val. Loss: 0.695 |  Val. Acc: 50.73%
	-------------------------------------------
	Train Loss: 0.694 | Train Acc: 51.76%
	 Val. Loss: 0.694 |  Val. Acc: 50.05%
	-------------------------------------------
	Train Loss: 0.696 | Train Acc: 49.85%
	 Val. Loss: 0.694 |  Val. Acc: 50.00%
	-------------------------------------------
	Train Loss: 0.695 | Train Acc: 49.41%
	 Val. Loss: 0.696 |  Val. Acc: 48.05%
	-------------------------------------------
	Train Loss: 0.695 | Train Acc: 50.73%
	 Val. Loss: 0.694 |  Val. Acc: 49.56%
	-------------------------------------------
	Train Loss: 0.695 | Train Acc: 49.71%
	 Val. Loss: 0.694 |  Val. Acc: 50.49%
	-------------------------------------------
	Train Loss: 0.6

	Train Loss: 0.693 | Train Acc: 50.10%
	 Val. Loss: 0.691 |  Val. Acc: 52.34%
	-------------------------------------------
	Train Loss: 0.693 | Train Acc: 50.68%
	 Val. Loss: 0.693 |  Val. Acc: 50.10%
	-------------------------------------------
	Train Loss: 0.693 | Train Acc: 50.78%
	 Val. Loss: 0.692 |  Val. Acc: 52.20%
	-------------------------------------------
	Train Loss: 0.693 | Train Acc: 50.59%
	 Val. Loss: 0.693 |  Val. Acc: 50.00%
	-------------------------------------------
	Train Loss: 0.693 | Train Acc: 51.46%
	 Val. Loss: 0.693 |  Val. Acc: 51.56%
	-------------------------------------------
	Train Loss: 0.694 | Train Acc: 50.29%
	 Val. Loss: 0.692 |  Val. Acc: 50.78%
	-------------------------------------------
	Train Loss: 0.694 | Train Acc: 49.61%
	 Val. Loss: 0.692 |  Val. Acc: 52.49%
	-------------------------------------------
	Train Loss: 0.693 | Train Acc: 51.46%
	 Val. Loss: 0.693 |  Val. Acc: 50.68%
	-------------------------------------------
	Train Loss: 0.6

KeyboardInterrupt: 

In [45]:
test(model, test_loader, config['device'])

0.5009765625